In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [3]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 8, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(47840, 674)
(47840,)
(39780, 674)
(39780,)


In [4]:
def average_rides_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    last_4_weeks_columns = [
            f"rides_t-{7*24}",  # 1 week ago
            f"rides_t-{14*24}", # 2 weeks ago
            f"rides_t-{21*24}", # 3 weeks ago
            f"rides_t-{28*24}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
    for col in last_4_weeks_columns:
        if col not in X.columns:
            raise ValueError(f"Missing required column: {col}")

    # Calculate the average of the last 4 weeks
    X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)

    return X

from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_4_weeks = FunctionTransformer(
    average_rides_last_4_weeks, validate=False
)

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_["hour"] = X_["pickup_hour"].dt.hour
        X_["day_of_week"] = X_["pickup_hour"].dt.dayofweek

        return X_.drop(columns=["pickup_hour", "pickup_location_id"])

add_temporal_features = TemporalFeatureEngineer()

In [6]:
import lightgbm as lgb

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
import numpy as np

# Step 1: Tune the learning rate
param_dist_lr = {
    'lgbmregressor__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3]
}

random_search_lr = RandomizedSearchCV(
    estimator=pipeline, param_distributions=param_dist_lr, n_iter=5, cv=3, scoring='neg_mean_absolute_error', random_state=42, verbose=2
)
random_search_lr.fit(X_train, y_train)

# Get the best learning rate
best_learning_rate = random_search_lr.best_params_['lgbmregressor__learning_rate']
print("Best Learning Rate:", best_learning_rate)

# Step 2: Tune other parameters using the best learning rate
param_dist = {
    'lgbmregressor__num_leaves': [31, 50, 70],
    'lgbmregressor__n_estimators': [100, 200, 500],
    'lgbmregressor__max_depth': [-1, 10, 20]
}

# Update the pipeline with the best learning rate
pipeline.set_params(lgbmregressor__learning_rate=best_learning_rate)

random_search = RandomizedSearchCV(
    estimator=pipeline, param_distributions=param_dist, n_iter=5, cv=3, scoring='neg_mean_absolute_error', random_state=42, verbose=2,n_jobs=1
)
random_search.fit(X_train, y_train)

# Get the best parameters and the best score
print("Best Parameters:", random_search.best_params_)
print("Best Score (Negative MAE):", random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Test Set MAE:", mae)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_child

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.193835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154557
[LightGBM] [Info] Number of data points in the train set: 31893, number of used features: 674
[LightGBM] [Info] Start training from score 13.775374
[CV] END ..................lgbmregressor__learning_rate=0.01; total time=   9.5s
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.146103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 151804
[LightGBM] [Info] Number of data points in the train set: 31893, number of used features: 674
[LightGBM] [Info] Start training from score 8.373217
[CV] END ..................lgbmregressor__learning_rate=0.01; total time=   6.6s
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing w

### Definition and Impact of Parameters in RandomizedSearchCV

In the `RandomizedSearchCV` cell, we define a parameter grid (`param_dist`) for hyperparameter tuning of the `LGBMRegressor`. Below are the definitions and impacts of each parameter:

1. **num_leaves**:
    - **Definition**: Maximum number of leaves in one tree.
    - **Impact**: Increasing `num_leaves` can improve the model's accuracy but may lead to overfitting. It also increases the model complexity and training time.

2. **learning_rate**:
    - **Definition**: Step size shrinkage used to prevent overfitting.
    - **Impact**: Lower values make the model more robust but require more trees. Higher values speed up training but may lead to overfitting.

3. **n_estimators**:
    - **Definition**: Number of boosting iterations (trees).
    - **Impact**: More trees can improve model performance but also increase training time and risk of overfitting.

4. **max_depth**:
    - **Definition**: Maximum depth of a tree.
    - **Impact**: Controls the complexity of the model. Deeper trees can capture more information but may overfit the data.

5. **min_child_samples**:
    - **Definition**: Minimum number of data points in a leaf node.
    - **Impact**: Helps control overfitting. Higher values prevent the model from learning overly specific patterns.

6. **subsample**:
    - **Definition**: Fraction of data to be used for fitting each tree.
    - **Impact**: Prevents overfitting by introducing randomness. Lower values can improve generalization but may reduce model accuracy.

7. **colsample_bytree**:
    - **Definition**: Fraction of features to be used for fitting each tree.
    - **Impact**: Reduces overfitting by introducing randomness. Lower values can improve generalization but may reduce model accuracy.

8. **reg_alpha**:
    - **Definition**: L1 regularization term on weights.
    - **Impact**: Adds a penalty for large coefficients, encouraging sparsity and reducing overfitting.

9. **reg_lambda**:
    - **Definition**: L2 regularization term on weights.
    - **Impact**: Adds a penalty for large coefficients, reducing overfitting but not encouraging sparsity.

By tuning these parameters, we aim to find the best combination that minimizes the mean absolute error (MAE) on the validation set, leading to a more accurate and generalizable model.

In [8]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
log_model_to_mlflow(random_search, X_test, "LGBMRegressorWFE_Hyper", "mean_absolute_error", score=mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.
INFO:src.experiment_utils:Experiment set to: LGBMRegressorWFE_Hyper
INFO:src.experiment_utils:Logged mean_absolute_error: 2.9697104087397532
D:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:M

2025/03/05 18:28:58 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Registered model 'RandomizedSearchCV' already exists. Creating a new version of this model...
2025/03/05 18:35:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomizedSearchCV, version 2
Created version '2' of model 'RandomizedSearchCV'.
INFO:src.experiment_utils:Model logged with name: RandomizedSearchCV


🏃 View run brawny-fish-411 at: https://dagshub.com/rockyglen/nyc_taxi.mlflow/#/experiments/6/runs/b11f38b3e4534f2b853de8ee862b0411
🧪 View experiment at: https://dagshub.com/rockyglen/nyc_taxi.mlflow/#/experiments/6
